In [11]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# from keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
from keras.layers import Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
# from keras.preprocessing.image import ImageDataGenerator
from keras.src.legacy.preprocessing.image import ImageDataGenerator

In [12]:
path="./traffic_Data/DATA"
labelfile="./labels.csv"
batch_size_val=16
steps_per_epoch_val=100
epochs_val=20
imageDimensions=(32,32,3)
testratio=0.1
validationratio=0.1

In [13]:
count=0
Images=[]
Classno=[]
mylist=os.listdir(path)
print("Total Classes Detected: ",len(mylist))
noofclasses=len(mylist)
print("Importing Classes .....")
for i in range(0,len(mylist)):
    mypics=os.listdir("./traffic_Data/DATA/" + str(count))
    for y in mypics:
        current=cv2.imread(path+"/"+str(count)+"/"+ y)
        current = cv2.resize(current,(100,100))
        Images.append(current)
        Classno.append(count)
    print(str(count) + "/" + str(noofclasses))
    count=count+1
print(str(noofclasses)+("/")+str(noofclasses))
print(" image Classno")
# print(Images)
# print(Classno)

Images=np.array(Images)
Classno=np.array(Classno)
print(Images.shape)
print(Classno.shape)

X_train, X_test, Y_train, Y_test = train_test_split(Images,Classno,test_size=testratio)
X_train, X_validation ,Y_train, Y_validation = train_test_split(X_train,Y_train,test_size=validationratio)

Total Classes Detected:  58
Importing Classes .....
0/58
1/58
2/58
3/58
4/58
5/58
6/58
7/58
8/58
9/58
10/58
11/58
12/58
13/58
14/58
15/58
16/58
17/58
18/58
19/58
20/58
21/58
22/58
23/58
24/58
25/58
26/58
27/58
28/58
29/58
30/58
31/58
32/58
33/58
34/58
35/58
36/58
37/58
38/58
39/58
40/58
41/58
42/58
43/58
44/58
45/58
46/58
47/58
48/58
49/58
50/58
51/58
52/58
53/58
54/58
55/58
56/58
57/58
58/58
 image Classno
(4170, 100, 100, 3)
(4170,)


In [14]:
################################################
print("DATA SHAPES")
print("Train:  ")
print(X_train.shape,Y_train.shape)
print("Validation:   ")
print(X_validation.shape, Y_validation.shape)
print("Test:  ")
print(X_test.shape,Y_test.shape)

data=pd.read_csv(labelfile)
print("data_shape",data.shape,type(data))

def grayscale(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(100,100)) 
    return img
def equalize(img):
    img=cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img=grayscale(img)
    img=equalize(img)
    img=img/255.0
    return img

DATA SHAPES
Train:  
(3377, 100, 100, 3) (3377,)
Validation:   
(376, 100, 100, 3) (376,)
Test:  
(417, 100, 100, 3) (417,)
data_shape (58, 2) <class 'pandas.core.frame.DataFrame'>


In [15]:
X_train=np.array(list(map(preprocessing,X_train)))
X_validation=np.array(list(map(preprocessing,X_validation)))
X_test=np.array(list(map(preprocessing,X_test)))
#cv2.imshow("Gray Scale Images: ", X_train[random.randint(0,len(X_train)-1)])
#cv2.waitKey(0)

X_train=X_train.reshape(X_train.shape[0],100,100,1)
X_validation=X_validation.reshape(X_validation.shape[0],100,100,1)
X_test=X_test.reshape(X_test.shape[0],100,100,1)

datagen=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)
datagen.fit(X_train)
batches=datagen.flow(X_train,Y_train,batch_size=20)
X_batch,Y_batch=next(batches)

Y_train=to_categorical(Y_train,noofclasses)
Y_validation=to_categorical(Y_validation,noofclasses)
Y_test=to_categorical(Y_test,noofclasses)

In [16]:
def mymodel():
    nooffilters=60
    sizeoffilters=(5,5)
    sizeoffilters2=(3,3)
    sizeofpool=(2,2)
    noofnodes=500
    model=Sequential()
    model.add((Conv2D(nooffilters,sizeoffilters,input_shape=(100,100,1),activation='relu')))
    model.add((Conv2D(nooffilters,sizeoffilters,activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeofpool))

    model.add((Conv2D(nooffilters//2,sizeoffilters2,activation='relu')))
    model.add((Conv2D(nooffilters//2,sizeoffilters2,activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeofpool))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(noofnodes,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noofclasses,activation='softmax'))

    model.compile(Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [18]:
###################33
model=mymodel()
history=model.fit(datagen.flow(X_train,Y_train,batch_size=batch_size_val),steps_per_epoch=steps_per_epoch_val,epochs=100,validation_data=(X_validation,Y_validation),shuffle=1)
score=model.evaluate(X_test,Y_test,verbose=0)
print('Test Score: ',score[0])
print('Test Accuracy: ',score[1])

D:\Study\9039 MACHINE LEARNING\Traffic Sign\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100


D:\Study\9039 MACHINE LEARNING\Traffic Sign\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 27s 251ms/step - accuracy: 0.0884 - loss: 3.7660 - val_accuracy: 0.2340 - val_loss: 3.1159
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 261ms/step - accuracy: 0.2136 - loss: 3.1639 - val_accuracy: 0.3723 - val_loss: 2.3812
Epoch 3/100
 12/100 ━━━━━━━━━━━━━━━━━━━━ 22s 250ms/step - accuracy: 0.2766 - loss: 2.8830

D:\Program Files\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.2761 - loss: 2.6191 - val_accuracy: 0.4309 - val_loss: 2.2787
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 258ms/step - accuracy: 0.3064 - loss: 2.7078 - val_accuracy: 0.5691 - val_loss: 1.6108
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 259ms/step - accuracy: 0.4274 - loss: 2.0893 - val_accuracy: 0.7154 - val_loss: 1.1424
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.4553 - loss: 1.6500 - val_accuracy: 0.7048 - val_loss: 1.1240
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 26s 256ms/step - accuracy: 0.5401 - loss: 1.7475 - val_accuracy: 0.7899 - val_loss: 0.9181
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 252ms/step - accuracy: 0.5993 - loss: 1.4419 - val_accuracy: 0.8245 - val_loss: 0.6957
Epoch 9/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.6229 - loss: 1.1581 - val_accuracy: 0.8271 - val_loss: 0.7424
Epoch 10/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 249ms/step - accuracy: 0.5991 - loss: 1.3309 - v

KeyboardInterrupt: 

In [ ]:

#################
# pickle_out=open("model_trained_avnmht.p","wb")
# pickle.dump(model,pickle_out)
# pickle_out.close()
#cv2.waitKey(0)